In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [313]:
with open('data.txt', 'r') as f:
    data = f.readlines()

# inputs
defense_ranks = [int(i) for i in data[0].split(',')] # d/st rankings (1-32) best = 1, worst = 32 
skill_score = [14.64 for i in range(len(defense_ranks))] # average skill score for RBs

# targets
targets = [float(i) for i in data[1].split(',')] # fantasy points scored by RBs against each d/st

# Training data
train_defense_ranks = np.array(defense_ranks[:10], dtype=np.float32)
train_skill_score = np.array(skill_score[:10], dtype=np.float32)
train_targets = np.array(targets[:10], dtype=np.float32)
 
# Testing data
test_defense_ranks = np.array(defense_ranks[10:], dtype=np.float32)
test_skill_score = np.array(skill_score[10:], dtype=np.float32)
test_targets = np.array(targets[10:], dtype=np.float32)

# Convert defense_ranks, skill_scores, and fantasy_scores to PyTorch tensors
input1 = torch.from_numpy(train_defense_ranks).view(-1, 1).float()
input2 = torch.from_numpy(train_skill_score).view(-1, 1).float()
test_input1 = torch.from_numpy(test_defense_ranks).view(-1, 1).float()
test_input2 = torch.from_numpy(test_skill_score).view(-1, 1).float()

targets = torch.from_numpy(train_targets).view(-1, 1).float()
test_targets = torch.from_numpy(test_targets).view(-1, 1).float()
# Concatenate inputs and skill_scores
inputs = torch.cat((input1, input2), dim=1)
test_inputs = torch.cat((test_input1, test_input2), dim=1)

In [385]:
# Define the neural network architecture
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Set hyperparameters
input_size = 2
hidden_size = 30
learning_rate = 0.001
num_epochs = 100

# Create the model
model = RegressionModel(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [425]:
# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss for every epoch
    # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Make predictions
with torch.no_grad():
    predicted = model(test_inputs)

print(f'loss = {loss.item()}\n') # add blank line
# Print the predictions and original scores
for i in range(len(test_inputs)):
    print(f"Defense Rank: {test_inputs[i][0]}, Predicted Fantasy Score: {predicted[i][0]}, Actual Fantasy Score: {test_targets[i].item()}")

loss = 18.259113311767578

Defense Rank: 1.0, Predicted Fantasy Score: 3.7792720794677734, Actual Fantasy Score: 5.599999904632568
Defense Rank: 10.0, Predicted Fantasy Score: 10.341328620910645, Actual Fantasy Score: 14.300000190734863
Defense Rank: 29.0, Predicted Fantasy Score: 17.85573959350586, Actual Fantasy Score: 7.599999904632568
Defense Rank: 15.0, Predicted Fantasy Score: 9.25361156463623, Actual Fantasy Score: 23.0
Defense Rank: 18.0, Predicted Fantasy Score: 10.529393196105957, Actual Fantasy Score: 7.699999809265137
